In [1]:
import PyPDF2
from tika import parser
from lxml import etree
import spacy
from spacy.lang.en import English
import bs4
import os
import pickle

In [2]:
# Tags to extract as paragraphs from the HTML text
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']

In [3]:
from nltk.corpus.reader.plaintext import CorpusReader
nlp = spacy.load("en_core_web_sm")

DOC_PATTERN = r'(?!\.)[\w\s\d\-]+\.pdf'
CAT_PATTERN = r'([\w_\s]+)/.*'
root = '/Users/nebo333/Desktop/Sample_text/FFIEC'

corpus = CorpusReader(root, DOC_PATTERN)

In [4]:
class PDFCorpusReader(CorpusReader):
    """
    A corpus reader for raw PDF documents to enable preprocessing.
    """

    def __init__(self, root, fileids=DOC_PATTERN, encoding='utf8', **kwargs):
        """
        Initialize the corpus reader. 
        Arguments are passed to the ``CorpusReader`` constructor.
        """
        
        # Initialize the NLTK corpus reader objects
        CorpusReader.__init__(self, root, fileids, encoding)
        
    def resolve(self, fileids):
        """
        Returns a list of fileids.
        """
        if fileids is not None:
            
            return self.fileids()
            
    def sizes(self, fileids=None):
        """
        Returns a list of tuples, the fileid and size on disk of the file.
        This function is used to detect oddly large files in the corpus.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids)

        # Create a generator, getting every path and computing filesize
        for path in self.abspaths(fileids):
            print(os.path.getsize(path))


In [5]:
import os

class Preprocessor(object):
    """
    The preprocessor wraps an `PDFCorpusReader` and performs tokenization
    and part-of-speech tagging.
    """
    def __init__(self, root, corpus, target=None, **kwargs):
        self.corpus = corpus
        self.target = target

    def fileids(self, fileids=None):
        fileids = self.corpus.resolve(fileids)
        if fileids:
            return fileids
        return self.corpus.fileids()
    
    def abspath(self, fileid):
        # Find the directory, relative to the corpus root.
        parent = os.path.relpath(
            os.path.dirname(self.corpus.abspath(fileid)), self.corpus.root
        )

        # Compute the name parts to reconstruct
        basename  = os.path.basename(fileid)
        name, ext = os.path.splitext(basename)

        # Create the pickle file extension
        basename  = name + '.pickle'

        # Return the path to the file relative to the target.
        return os.path.normpath(os.path.join(self.target, parent, basename))
    
    def create_doc_struc(self, root, fileid):
        
        file_id = root + '/' + fileid
        file_data = parser.from_file(file_id)
        # Get files text content
        content_text = file_data['content']
        # Create doc object
        doc = nlp(content_text)
        
        doc_ls = []
        for i in doc.sents:
            sent_ls=[]
            for j in i:
                if j.is_stop == False:
                    if j.is_alpha == True:
                        token = j.text
                        pos = j.pos_
                        tup = (token,pos)
                        sent_ls.append(tup)
            doc_ls.append(sent_ls)
        yield doc_ls
            
    def process(self, fileid):
        """
        For a single file, checks the location on disk to ensure no errors,
        uses +tokenize()+ to perform the preprocessing, and writes transformed
        document as a pickle to target location.
        """
        # Compute the outpath to write the file to.
        target = self.abspath(fileid)
        parent = os.path.dirname(target)

        # Make sure the directory exists
        if not os.path.exists(parent):
            os.makedirs(parent)

        # Make sure that the parent is a directory and not a file
        if not os.path.isdir(parent):
            raise ValueError(
                "Please supply a directory to write preprocessed data to."
            )

        # Create a data structure for the pickle
        document = list(self.create_doc_struc(root,fileid))

        # Open and serialize the pickle to disk
        with open(target, 'wb') as f:
            pickle.dump(document, f, pickle.HIGHEST_PROTOCOL)

        # Clean up the document
        del document

        # Return the target fileid
        return target
    
    def transform(self, fileids=None):
        # Make the target directory if it doesn't already exist
        if not os.path.exists(self.target):
            os.makedirs(self.target)

        # Resolve the fileids to start processing
        for fileid in self.fileids(fileids):
            yield self.process(fileid)

In [6]:
myreader = PDFCorpusReader(root)

In [7]:
target = '/Users/nebo333/Desktop/Sample_text/pickle_files'

In [8]:
preproc = Preprocessor(root, myreader, target)

In [9]:
ids = myreader.fileids()

In [10]:
for i in preproc.create_doc_struc(root,ids[0]):
    print(i)

[[('Table', 'NOUN'), ('ContentsTable', 'PROPN'), ('ContentsTable', 'PROPN'), ('ContentsTable', 'PROPN'), ('Contents', 'NOUN')], [('IntroductionIntroductionIntroductionIntroduction', 'NOUN')], [('Audit', 'PROPN'), ('Roles', 'PROPN'), ('ResponsibilitiesIT', 'ADJ'), ('Audit', 'PROPN'), ('Roles', 'PROPN'), ('ResponsibilitiesIT', 'ADJ'), ('Audit', 'PROPN'), ('Roles', 'PROPN'), ('ResponsibilitiesIT', 'ADJ'), ('Audit', 'PROPN'), ('Roles', 'PROPN'), ('Responsibilities', 'PROPN')], [('Board', 'PROPN'), ('Directors', 'PROPN'), ('Senior', 'PROPN'), ('Management', 'PROPN')], [('Audit', 'PROPN'), ('Management', 'PROPN')], [('Internal', 'PROPN'), ('Audit', 'PROPN'), ('Staff', 'PROPN')], [('Operating', 'VERB'), ('Management', 'NOUN')], [('External', 'PROPN'), ('Auditors', 'NOUN'), ('Independence', 'NOUN'), ('Staffing', 'NOUN'), ('Internal', 'PROPN'), ('AuditIndependence', 'PROPN'), ('Staffing', 'PROPN'), ('Internal', 'PROPN'), ('AuditIndependence', 'PROPN'), ('Staffing', 'PROPN'), ('Internal', 'PROPN

In [11]:
for i in preproc.transform(ids):
    print(i)

/Users/nebo333/Desktop/Sample_text/pickle_files/ffiec_itbooklet_audit.pickle
/Users/nebo333/Desktop/Sample_text/pickle_files/ffiec_itbooklet_businesscontinuitymanagement.pickle
/Users/nebo333/Desktop/Sample_text/pickle_files/ffiec_itbooklet_developmentandacquisition.pickle


In [12]:
PKL_PATTERN = r'(?!\.)[\w\s\d\-]+\.pickle'

class PickledCorpusReader(PDFCorpusReader):

    def __init__(self, root, fileids=PKL_PATTERN, **kwargs):
        CorpusReader.__init__(self, root, fileids)

    def docs(self, fileids=None):
        fileids = self.resolve(fileids)
        # Load one pickled document into memory at a time.
        for path in self.abspaths(fileids):
            with open(path, 'rb') as f:
                yield pickle.load(f)[0]
    
    def paras(self, fileids=None):
        for doc in self.docs(fileids):
            for para in doc:
                yield para
                
    def sents(self, fileids=None):
        for para in self.paras(fileids):
            for sent in para:
                yield sent
                
    def tagged(self, fileids=None):
        for sent in self.sents(fileids):
            for tagged_token in sent:
                yield tagged_token

    def words(self, fileids=None):
        for tagged in self.tagged(fileids):
            yield tagged[0]
    
    ## Need to add in sentence, tuple and token yield functions


In [13]:
pic_reader = PickledCorpusReader(target)

In [14]:
ids

['ffiec_itbooklet_audit.pdf',
 'ffiec_itbooklet_businesscontinuitymanagement.pdf',
 'ffiec_itbooklet_developmentandacquisition.pdf']

In [19]:
doc_ls = []
for i in pic_reader.paras('ffiec_itbooklet_businesscontinuitymanagement.pdf'):
    para_ls=[]
    for j in i:
        wordj[0])

Table
ContentsTable
ContentsTable
ContentsTable
Contents
IntroductionIntroductionIntroductionIntroduction
Audit
Roles
ResponsibilitiesIT
Audit
Roles
ResponsibilitiesIT
Audit
Roles
ResponsibilitiesIT
Audit
Roles
Responsibilities
Board
Directors
Senior
Management
Audit
Management
Internal
Audit
Staff
Operating
Management
External
Auditors
Independence
Staffing
Internal
AuditIndependence
Staffing
Internal
AuditIndependence
Staffing
Internal
AuditIndependence
Staffing
Internal
Audit
Independence
Staffing
Internal
Audit
ProgramInternal
Audit
ProgramInternal
Audit
ProgramInternal
Audit
Program
Risk
Assessment
Risk
Based
AuditingRisk
Assessment
Risk
Based
AuditingRisk
Assessment
Risk
Based
AuditingRisk
Assessment
Risk
Based
Auditing
Program
Elements
Risk
Scoring
System
Audit
Participation
Application
Development
Acquisition
Conversions
TestingAudit
Participation
Application
Development
Acquisition
Conversions
TestingAudit
Participation
Application
Development
Acquisition
Conversions
TestingAu

institutions
complex
operations
benefits
derived
time
manager
internal
audit
audit
staff
likely
outweigh
cost
small
institutions
employees
simple
operations
costs
outweigh
benefits
institution
internal
auditor
ensure
maintains
objective
independent
internal
function
implementing
comprehensive
internal
reviews
significant
internal
controls
key
characteristic
reviews
directing
performing
review
responsible
managing
operating
controls
StaffingStaffingStaffingStaffing
Personnel
performing
audits
information
systems
knowledge
commensurate
scope
sophistication
institution
environment
possess
sufficient
analytical
skills
determine
report
root
cause
deficiencies
internal
expertise
inadequate
board
consider
qualified
external
sources
management
consultants
independent
auditors
professionals
supplement
perform
institution
internal
audit
function
institutions
person
group
responsibilities
outside
audit
function
performs
audits
Generally
institutions
approach
centralize
audit
coverage
assign
audit

data
impact
risk
assessment
scoring
Accordingly
operating
management
required
auditors
date
major
changes
departments
functions
introduction
new
product
implementation
new
system
application
conversions
significant
changes
organization
staff
Audit
Participation
Application
Development
Audit
Participation
Application
Development
Audit
Participation
Application
Development
Audit
Participation
Application
Development
Acquisition
Conversions
TestingAcquisition
Conversions
TestingAcquisition
Conversions
TestingAcquisition
Conversions
Testing
Action
SummaryAction
SummaryAction
SummaryAction
Summary
Senior
management
involve
audit
major
application
development
acquisition
conversion
testing
development
acquisition
conversion
automated
application
lengthy
complex
process
requiring
significant
degree
interaction
programming
staff
user
departments
internal
audit
process
known
system
development
life
cycle
system
development
methodology
requires
detailed
developmental
stages
ensure
applications
m

verify
internal
control
environment
TSP
use
additional
verification
monitoring
procedures
discussed
fully
Outsourcing
Technology
Services
Booklet
FFIEC
Examination
Handbook
Refer
booklet
additional
information
vendor
management
supplement
examination
coverage
booklet
Audit
Booklet
Page
EndnotesEndnotesEndnotesEndnotes
booklet
uses
terms
institution
financial
institution
describe
insured
banks
thrifts
credit
unions
technology
service
providers
provide
services
entities
Board
Governors
Federal
Reserve
System
Federal
Reserve
Board
Federal
Deposit
Insurance
Corporation
FDIC
National
Credit
Union
Administration
NCUA
Office
Comptroller
Currency
OCC
Office
Thrift
Supervision
OTS
include
Interagency
Policy
Statement
Internal
Audit
Function
Outsourcing
March
Interagency
Policy
Statement
External
Auditing
Programs
Banks
Savings
Associations
September
Interagency
Policy
Statement
Coordination
Communication
External
Auditors
Examiners
July
federal
credit
union
board
directors
required
establish
su

auditor
qualifications
Determine
outsourcing
arrangement
maintains
improves
quality
internal
audit
function
institution
internal
controls
examiner
Review
performance
contractual
criteria
audit
vendor
internal
evaluations
audit
vendor
Review
outsourced
internal
audit
reports
sample
audit
work
papers
Determine
adequate
prepared
accordance
audit
program
outsourcing
agreement
Determine
work
papers
disclose
specific
program
steps
calculations
evidence
support
procedures
conclusions
set
forth
outsourced
reports
Determine
scope
outsourced
internal
audit
procedures
adequate
Determine
key
employees
institution
audit
vendor
clearly
understand
lines
communication
internal
control
problems
matters
noted
audit
vendor
internal
audits
addressed
Determine
management
audit
vendor
revises
scope
outsourced
audit
work
appropriately
institution
environment
activities
risk
exposures
systems
change
significantly
Audit
Booklet
Page
Determine
directors
ensure
institution
effectively
manages
outsourced
internal

...

tools
store
descriptions
structure
format
data
data
tables
Advanced
data
dictionaries
store
source
code
copies
field
record
code
descriptions
use
software
design
development
activities
Primary
issues
consider
reviewing
design
configuration
database
management
systems
include
access
controls
auditing
features
Management
restrict
direct
privileged
access
database
opposed
accessing
information
application
authorized
personnel
DBMS
journaling
feature
allows
organizations
track
data
changes
Journaling
provides
audit
trails
data
changes
facilitates
safe
recovery
data
errors
occur
available
organizations
employ
automated
auditing
tools
journaling
identify
accessed
attempted
access
database
data
changed
DBMS
validate
users
record
row
levels
log
activities
detailed
validation
levels
provide
strong
security
controls
Examiners
consider
validation
levels
assessing
adequacy
DBMS
controls
Strong
DBMS
controls
include
data
change
logs
input
validity
checks
locking
rollback
mechanisms
ability
recover


unauthorized
software
use
copyright
violations
measures
help
prevent
copyright
violations
best
control
mechanism
strict
corporate
policy
management
auditors
communicate
enforce
Management
uncompromising
attitude
copyright
violations
organization
security
administrator
responsible
monitoring
enforcing
policy
Software
Development
Specifications
Performance
StandardsSoftware
Development
Specifications
Performance
StandardsSoftware
Development
Specifications
Performance
StandardsSoftware
Development
Specifications
Performance
Standards
Contracts
development
custom
software
describe
define
expected
performance
attributes
functionality
software
contract
describe
equipment
required
operate
software
ensure
appropriate
compatibility
Vendors
required
meet
exceed
institution
internal
development
policies
standards
opening
negotiations
issuing
request
proposal
custom
software
development
organizations
clear
idea
essential
business
needs
addressed
software
adequate
understanding
organization
presen

tested
programs
Management
establish
program
approval
standards
include
procedures
verifying
test
results
inspecting
modified
code
confirming
source
object
codes
match
Emergency
ModificationsEmergency
ModificationsEmergency
ModificationsEmergency
Modifications
Development
Acquisition
Booklet
Page
Emergency
modifications
periodically
needed
correct
software
problems
restore
processing
operations
quickly
changes
completed
quickly
implemented
controlled
manner
Emergency
change
standards
include
procedures
similar
routine
change
controls
standards
include
abbreviated
change
request
evaluation
approval
procedures
ensure
changes
quickly
standards
designed
ensure
management
completes
detailed
evaluations
documentation
emergency
changes
soon
possible
implementation
possible
emergency
changes
tested
prior
implementation
management
unable
thoroughly
test
emergency
modifications
installation
critical
appropriately
backup
files
programs
established
procedures
place
Appropriate
backups
established


responsibilities
concerning
technology
systems
initiatives
Evaluate
organizational
responsibilities
ensure
board
management
Development
Acquisition
Booklet
Page
Clearly
define
appropriately
assign
responsibilities
Appropriately
assign
security
audit
quality
assurance
personnel
related
projects
Establish
appropriate
segregation
duty
compensating
controls
Establish
appropriate
project
technology
committee
board
reporting
requirements
Objective
Assess
level
characteristics
risks
associated
development
Objective
Assess
level
characteristics
risks
associated
development
Objective
Assess
level
characteristics
risks
associated
development
Objective
Assess
level
characteristics
risks
associated
development
acquisition
maintenance
activities
materially
impact
maintenance
activities
materially
impact
maintenance
activities
materially
impact
maintenance
activities
materially
impact
organization
Assess
risks
identified
objectives
evaluate
adequacy
risk
management
programs
Risk
identification
asses

availability
FlowchartsFlowchartsFlowchartsFlowcharts
Traditional
flowcharts
involve
use
geometric
symbols
diamonds
ovals
rectangles
represent
sequencing
program
logic
Software
packages
available
automatically
chart
programs
enable
programmer
chart
program
need
draw
manually
Functional
requirementsFunctional
requirementsFunctional
requirementsFunctional
requirements
business
operational
security
features
organization
wants
included
program
IterativeIterativeIterativeIterative
Repetitive
cyclical
Iterative
software
development
involves
completion
project
tasks
phases
repetitive
cycles
Tasks
phase
activities
repeated
desired
result
achieved
LAN
Acronym
Local
Area
Network
MetricMetricMetricMetric
quantitative
measurement
MilestoneMilestoneMilestoneMilestone
major
project
event
NetworkNetworkNetworkNetwork
computer
systems
grouped
share
information
Development
Acquisition
Booklet
Page
software
hardware
Object
codeObject
codeObject
codeObject
code
Software
program
instructions
compiled
tran

In [20]:
for i in pic_reader.paras('ffiec_itbooklet_businesscontinuitymanagement.pdf'):
    print(i)

[('Table', 'NOUN'), ('ContentsTable', 'PROPN'), ('ContentsTable', 'PROPN'), ('ContentsTable', 'PROPN'), ('Contents', 'NOUN')]
[('IntroductionIntroductionIntroductionIntroduction', 'NOUN')]
[('Audit', 'PROPN'), ('Roles', 'PROPN'), ('ResponsibilitiesIT', 'ADJ'), ('Audit', 'PROPN'), ('Roles', 'PROPN'), ('ResponsibilitiesIT', 'ADJ'), ('Audit', 'PROPN'), ('Roles', 'PROPN'), ('ResponsibilitiesIT', 'ADJ'), ('Audit', 'PROPN'), ('Roles', 'PROPN'), ('Responsibilities', 'PROPN')]
[('Board', 'PROPN'), ('Directors', 'PROPN'), ('Senior', 'PROPN'), ('Management', 'PROPN')]
[('Audit', 'PROPN'), ('Management', 'PROPN')]
[('Internal', 'PROPN'), ('Audit', 'PROPN'), ('Staff', 'PROPN')]
[('Operating', 'VERB'), ('Management', 'NOUN')]
[('External', 'PROPN'), ('Auditors', 'NOUN'), ('Independence', 'NOUN'), ('Staffing', 'NOUN'), ('Internal', 'PROPN'), ('AuditIndependence', 'PROPN'), ('Staffing', 'PROPN'), ('Internal', 'PROPN'), ('AuditIndependence', 'PROPN'), ('Staffing', 'PROPN'), ('Internal', 'PROPN'), ('Au

[('level', 'NOUN'), ('formality', 'NOUN'), ('BCM', 'PROPN'), ('ERM', 'PROPN'), ('integration', 'NOUN'), ('commensurate', 'VERB'), ('entity', 'NOUN'), ('complexity', 'NOUN'), ('risk', 'NOUN'), ('profile', 'NOUN')]
[('Figure', 'NOUN'), ('Business', 'PROPN'), ('Continuity', 'PROPN'), ('Management', 'PROPN'), ('Elements', 'PROPN'), ('Relative', 'ADJ'), ('Enterprise', 'PROPN'), ('Risk', 'PROPN'), ('Management', 'PROPN')]
[('Refer', 'VERB'), ('Department', 'PROPN'), ('Treasury', 'PROPN'), ('Department', 'PROPN'), ('Homeland', 'PROPN'), ('Security', 'PROPN')]
[('DHS', 'PROPN')]
[('Financial', 'PROPN'), ('Services', 'PROPN'), ('Sector', 'PROPN'), ('Specific', 'PROPN'), ('Plan', 'PROPN')]
[('ERM', 'PROPN'), ('process', 'NOUN'), ('effected', 'VERB'), ('entity', 'NOUN'), ('board', 'NOUN'), ('directors', 'NOUN'), ('management', 'NOUN'), ('personnel', 'NOUN'), ('applied', 'VERB'), ('strategy', 'NOUN'), ('setting', 'VERB'), ('enterprise', 'NOUN'), ('designed', 'VERB'), ('identify', 'VERB'), ('potent

[('Data', 'NOUN'), ('repository', 'NOUN'), ('maintenance', 'NOUN')]
[('Protect', 'PROPN'), ('offline', 'ADJ'), ('data', 'NOUN'), ('backups', 'NOUN'), ('destructive', 'ADJ'), ('malware', 'NOUN'), ('corrupt', 'VERB'), ('production', 'NOUN'), ('online', 'ADJ'), ('backup', 'ADJ'), ('versions', 'NOUN'), ('data', 'NOUN')]
[('FFIEC', 'PROPN'), ('Examination', 'PROPN'), ('Handbook', 'PROPN')]
[('Business', 'PROPN'), ('Continuity', 'PROPN'), ('Management', 'PROPN')]
[('November', 'PROPN')]
[('Determine', 'VERB'), ('management', 'NOUN'), ('documented', 'VERB'), ('implemented', 'VERB'), ('appropriate', 'ADJ'), ('following', 'VERB'), ('resilience', 'NOUN'), ('measures', 'NOUN'), ('personnel', 'NOUN')]
[('Staffing', 'NOUN'), ('skills', 'NOUN'), ('needed', 'VERB'), ('operate', 'VERB'), ('critical', 'ADJ'), ('functions', 'NOUN'), ('related', 'VERB'), ('business', 'NOUN'), ('continuity', 'NOUN')]
[('Lodging', 'PROPN'), ('arrangements', 'NOUN'), ('displaced', 'ADJ'), ('employees', 'NOUN'), ('families',

[('Repeatable', 'ADJ'), ('Organizations', 'NOUN'), ('Repeatable', 'ADJ'), ('capabilities', 'NOUN'), ('use', 'VERB'), ('fundamental', 'ADJ'), ('project', 'NOUN')]
[('planning', 'NOUN'), ('scheduling', 'NOUN'), ('monitoring', 'NOUN'), ('procedures', 'NOUN')]
[('Defined', 'VERB'), ('Organizations', 'NOUN'), ('Defined', 'VERB'), ('capabilities', 'NOUN'), ('use', 'VERB'), ('approved', 'VERB'), ('formalized', 'VERB'), ('management', 'NOUN'), ('development', 'NOUN'), ('standards', 'NOUN'), ('procedures', 'NOUN'), ('customized', 'VERB'), ('meet', 'VERB'), ('specific', 'ADJ'), ('project', 'NOUN'), ('requirements', 'NOUN'), ('projects', 'NOUN')]
[('Managed', 'VERB'), ('Organizations', 'NOUN'), ('Managed', 'VERB'), ('capabilities', 'NOUN'), ('measure', 'NOUN'), ('understand', 'VERB'), ('control', 'NOUN'), ('development', 'NOUN'), ('procedures', 'NOUN'), ('product', 'NOUN'), ('quality', 'NOUN')]
[('Optimizing', 'VERB'), ('Organizations', 'NOUN'), ('Optimized', 'ADJ'), ('capabilities', 'NOUN'), ('u